# Clasificadores Scikit-Learn

https://scikit-learn.org/stable/

### Etapas en el ajuste de una máquina de aprendizaje

1. Selección de características
2. Selección de una métrica de desempeño
3. Selección de un clasificador y su algoritmo de optimización
4. Evaluar el desempeño del modelo
5. Afinamiento y ajustes de los parámetros del algoritmo.

In [ ]:
# Conjuntos de datos almacenados en Toolbox
from sklearn import datasets  
# Separación del conjunto de datos en conjunto de entrenamiento y test
from sklearn.model_selection import train_test_split 
# Método para reescalar los datos
from sklearn.preprocessing import StandardScaler
# Paquete numérico para el procesamiento de datos
import numpy as np
# Paquete para realizar los gráficos 
import matplotlib.pyplot as plt
%matplotlib inline

### Datos del IRIS
Corresponde a una especie de planta con 3 tipos de caregorías: setosa, versicolor y virgínica.
A cada planta se le midieron características de estructuras: logitud y ancho del sépalo, y longitud y ancho del pétalo. 
https://archive.ics.uci.edu/ml/datasets/Iris

<img src="iris.png">

In [ ]:
iris = datasets.load_iris()
# Para los datos de entrada sólo utilizaremos 2 atributos 
# con el fin de poder graficar los resultados
X = iris.data[:, [2,3]]
y = iris.target

In [ ]:
# Gráfico de dispersión para obervar los datos que se tienen
d_clase0 = X[y==0]
d_clase1 = X[y==1]
d_clase2 = X[y==2]
etiquetas = np.unique(y)
plt.plot(d_clase0[:,0],d_clase0[:,1], 'r.')
plt.plot(d_clase1[:,0],d_clase1[:,1], 'g.')
plt.plot(d_clase2[:,0],d_clase2[:,1], 'b.')
plt.legend(iris.target_names)
plt.title("Gráfico de dispersión de los datos del iris")
plt.xlabel(iris.feature_names[2])
plt.ylabel(iris.feature_names[3])

In [ ]:
#import seaborn as sns
#sns.scatterplot(x=X[:,0], y=X[:,1], hue=y)
sns.__version__

**Separación de los datos en conjunto de entrenamiento y test**

In [ ]:
X_entrenamiento, X_test, y_entrenamiento, y_test = train_test_split(X, y, 
                        test_size = 0.4, random_state=0)

**Reescalamiento de las características**

Con el fin de que todos los atributos tenga igual importancia al entrenar un clasificador, y debido a que los clasificadores están optimizados para los datos escalados. Procederemos a reescalar los datos.

En particular, el escalamiento que se aplicará es centrar los datos en 0 y cambiar su variabilidad a 1:

$$Z=\frac{X - \overline{X}}{S_{X}}$$

donde $\overline{X}$ corresponde al promedio de los datos y $S_{X}$ a la desviación estándar de los datos de entrada.

In [ ]:
sc = StandardScaler()
# Los datos son reescalado con respecto a los datos de entrenamiento
sc.fit(X_entrenamiento) 
# Tanto el conjunto de entrenamiento como el de test son reescalados
Z_entrenamiento = sc.transform(X_entrenamiento)
Z_test = sc.transform(X_test)

### Construcción del clasificador LDA (Linear Discriminant Analysis)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
clasificador1 = LinearDiscriminantAnalysis()
clasificador1.fit(Z_entrenamiento,y_entrenamiento)
y_pred1 = clasificador1.predict(Z_test)

### Construcción del clasificador QDA (Quadratic Discriminant Analysis)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
clasificador2 = QuadraticDiscriminantAnalysis()
clasificador2.fit(Z_entrenamiento,y_entrenamiento)
y_pred2 = clasificador2.predict(Z_test)

### Construcción de un clasificador SVC (Support Vector Classification)

In [ ]:
from sklearn.svm import SVC
clasificador3=SVC(kernel='linear', C=0.025)
clasificador3.fit(Z_entrenamiento,y_entrenamiento)
y_pred3 = clasificador3.predict(Z_test)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
confusion_matrix(y_test, y_pred3)
plot_confusion_matrix(y_test, y_pred3, classes = iris.target_names, title='Confusion matrix, without normalization')

In [ ]:
# Gráfico de dispersión para obervar los datos que se tienen
d_err_clas = X_test[y_pred3!=y_test]
d_clase0 = X[y==0]
d_clase1 = X[y==1]
d_clase2 = X[y==2]
etiquetas = np.unique(y)
plt.plot(d_clase0[:,0],d_clase0[:,1], 'r.')
plt.plot(d_clase1[:,0],d_clase1[:,1], 'g.')
plt.plot(d_clase2[:,0],d_clase2[:,1], 'b.')
plt.scatter(d_err_clas[:,0],d_err_clas[:,1], c='k')
plt.legend(iris.target_names)
plt.title("Gráfico de dispersión de los datos del iris")
plt.xlabel(iris.feature_names[2])
plt.ylabel(iris.feature_names[3])